### Memory Management

#### Memory is a service that provides a long-term knowledge storage for the agents.

Session = Short-term memory (single conversation)(like application state - temporary)

Memory = Long-term knowledge (across multiple conversations)(like database - persistent)

In [57]:
import os

try:
  GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
  print("Setup complete.")
except Exception as e:
  print("Error: ",e)


Setup complete.


In [58]:
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.memory import InMemoryMemoryService
from google.adk.tools import load_memory, preload_memory
from google.genai import types

print("ADK components imported successfully.")

ADK components imported successfully.


In [59]:
async def run_session(
    runner_instance:Runner,
    user_queries: list[str] |str,
    session_id: str="default"
):
  """Helper function to run queries in a session and display responses."""
  print(f"\n ### Session:{session_id}")

  try:
    session = await session_service.create_session(
      app_name=APP_NAME,user_id=USER_ID,session_id = session_id
    )
  except:
    session = await session_service.get_session(
      app_name = APP_NAME, user_id=USER_ID,session_id= session_id
    )

  #Convert single query to list
  if isinstance(user_queries,str):
    user_queries = [user_queries]

  #process each query
  for query in user_queries:
    print(f"\n User>{query}")
    query_content = types.Content(role="user",parts=[types.Part(text=query)])

    #stream agent response
    async for event in runner_instance.run_async(
      user_id=USER_ID,session_id=session_id,new_message=query_content
    ):
      if event.is_final_response() and event.content and event.content.parts:
        text = event.content.parts[0].text
        if text and text != "None":
          print(f"Model: > {text}")

### Memory Workflow

1. initialize -> create a `MemoryService` & provide it to the agent via `Runner`
2. ingest -> transfer session data to memory using `add_session_to_memory()`
3. retrieve -> search stored memories using `search_memory()`

In [60]:
memory_service = (
  InMemoryMemoryService() #adk's built in memory service for dev and testing
)                         # creates the space into the RAM

In [61]:
retry_config = types.HttpRetryOptions(
  exp_base=7,
  initial_delay=1,
  attempts=5,
  http_status_codes=[429,500,503,504]
)

In [62]:
APP_NAME = "MemoryDemoApp"
USER_ID = "demo_user"

user_agent = LlmAgent(
  model=Gemini(model="gemini-2.5-flash-lite",retry_options=retry_config),
  name="MemoryDemoAgent",
  instruction="Answer user questions in simple words."
)


In [63]:
session_service = InMemorySessionService() #handles conversations

runner = Runner(
  agent=user_agent,
  app_name= APP_NAME,
  session_service=session_service,
  memory_service= memory_service,
)

App name mismatch detected. The runner is configured with app name "MemoryDemoApp", but the root agent was loaded from "/Users/liakooras/Desktop/UDEMIES/genAiKaggle/.conda/lib/python3.12/site-packages/google/adk/agents", which implies app name "agents".


Adding `memory_service` to the `runner` makes memory available to the agent, but doesnt automatically uses it. It has to:
1. ingest data using `add_session_to_memory()`
2. enable retrieval by giving the agent memory tools (`load_memory` or `preload_memory`)

In [64]:
await run_session(
  runner,
  "My favourite moto is 'Nothing easy is worth doing', can you think of an alternative way of expressing it? ",
  "conversation-01" #sessionID
)


 ### Session:conversation-01

 User>My favourite moto is 'Nothing easy is worth doing', can you think of an alternative way of expressing it? 
Model: > That's a great motto! Here are a few alternative ways to say "Nothing easy is worth doing," using simpler words:

*   **Hard things are the most rewarding.**
*   **If it's easy, it's probably not that important.**
*   **The best things take a lot of effort.**
*   **Don't shy away from a challenge; that's where the good stuff is.**
*   **Real achievements don't come easily.**


In [65]:
#Lets verify the conversation was captured in the session

session = await session_service.get_session(
  user_id=USER_ID, app_name=APP_NAME,session_id="conversation-01")

In [66]:
session.events[0]

Event(model_version=None, content=Content(
  parts=[
    Part(
      text="My favourite moto is 'Nothing easy is worth doing', can you think of an alternative way of expressing it? "
    ),
  ],
  role='user'
), grounding_metadata=None, partial=None, turn_complete=None, finish_reason=None, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=None, live_session_resumption_update=None, input_transcription=None, output_transcription=None, avg_logprobs=None, logprobs_result=None, cache_metadata=None, citation_metadata=None, invocation_id='e-448176eb-4359-478f-ac2f-52611b453946', author='user', actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}, requested_tool_confirmations={}, compaction=None, end_of_agent=None, agent_state=None, rewind_before_invocation_id=None), long_running_tool_ids=None, branch=None, id='bef9d0bf-c1d1-4298-87e8-b8677bd751d2', timestamp=1

In [67]:
print("Session Contains:")

for event in session.events:
  text = (
    event.content.parts[0].text[0:200]
    if event.content and event.content.parts
    else "empty"
  )
  print(f"{event.content.role} : {text}")

Session Contains:
user : My favourite moto is 'Nothing easy is worth doing', can you think of an alternative way of expressing it? 
model : That's a great motto! Here are a few alternative ways to say "Nothing easy is worth doing," using simpler words:

*   **Hard things are the most rewarding.**
*   **If it's easy, it's probably not that


In [68]:
# transfering the conversation into memory with add_session_to_memory()

await memory_service.add_session_to_memory(session)

In [69]:
# adding load memory tool to the agent

user_agent = LlmAgent(
  model=Gemini(model="gemini-2.5-flash-lite",retry_options=retry_config),
  name = "MemoryDemoAgent",
  description="Answer user questions in simple words. Use load_memory tool if you need to recall past conversations.",
  tools=[load_memory]
)

In [70]:
# updating the runner also
runner = Runner(
  agent=user_agent,
  app_name=APP_NAME,
  session_service=session_service,
  memory_service=memory_service,
)

App name mismatch detected. The runner is configured with app name "MemoryDemoApp", but the root agent was loaded from "/Users/liakooras/Desktop/UDEMIES/genAiKaggle/.conda/lib/python3.12/site-packages/google/adk/agents", which implies app name "agents".


In [71]:
await run_session(runner,"Whats my favourite motto?","motto test")


 ### Session:motto test

 User>Whats my favourite motto?


In [77]:
# Complete manual workflow test

await run_session(runner,"My Birthday is on July 31.","birthday-session-01")


 ### Session:birthday-session-01

 User>My Birthday is on July 31.
Model: > I've noted that your birthday is on July 31.


In [78]:
birthday_session = await session_service.get_session(user_id=USER_ID,app_name=APP_NAME,session_id="birthday-session-01")


In [79]:
await memory_service.add_session_to_memory(birthday_session)

In [80]:
#test

await run_session(runner, "When is my birthday?","birthday-session-02")


 ### Session:birthday-session-02

 User>When is my birthday?


Model: > Your birthday is on July 31.


In [84]:
# Manual Memory Search

#Search for color preferences

search_response = await memory_service.search_memory(
  app_name=APP_NAME,user_id=USER_ID,query="What the user's favourite motto?"
)

In [85]:
search_response.memories

[MemoryEntry(content=Content(
   parts=[
     Part(
       text="My favourite moto is 'Nothing easy is worth doing', can you think of an alternative way of expressing it? "
     ),
   ],
   role='user'
 ), author='user', timestamp='2025-11-21T12:43:23.443404'),
 MemoryEntry(content=Content(
   parts=[
     Part(
       text="""That's a great motto! Here are a few alternative ways to say "Nothing easy is worth doing," using simpler words:
 
 *   **Hard things are the most rewarding.**
 *   **If it's easy, it's probably not that important.**
 *   **The best things take a lot of effort.**
 *   **Don't shy away from a challenge; that's where the good stuff is.**
 *   **Real achievements don't come easily.**"""
     ),
   ],
   role='model'
 ), author='MemoryDemoAgent', timestamp='2025-11-21T12:43:23.444550')]

In [88]:
print("Search Results:")
print(f" Found {len(search_response.memories)} relevant memories.")

Search Results:
 Found 2 relevant memories.


In [107]:
for memory in search_response.memories:
  if memory.content and memory.content.parts:
    text = memory.content.parts[0].text[0:100]
    print(f"[{memory.author}] : {text}...")

[user] : My favourite moto is 'Nothing easy is worth doing', can you think of an alternative way of expressin...
[MemoryDemoAgent] : That's a great motto! Here are a few alternative ways to say "Nothing easy is worth doing," using si...


In [115]:
#test2

await run_session(runner,"My favourite team is PAOK Salonica.","club-conversation-1")


 ### Session:club-conversation-1

 User>My favourite team is PAOK Salonica.


Model: > I see. You mentioned that PAOK Salonica is your favourite team.


In [116]:
team_session = await session_service.get_session(user_id=USER_ID,app_name=APP_NAME,session_id="club-conversation")

In [117]:
await memory_service.add_session_to_memory(team_session)

In [120]:
await run_session(runner,"Whats my favourite football club?","club-conversation-2")


 ### Session:club-conversation-2

 User>Whats my favourite football club?


In [121]:
user_agent = LlmAgent(
  model=Gemini(model="gemini-2.5-flash-lite",retry_options=retry_config),
  name = "MemoryDemoAgent",
  description="Answer user questions in simple words. Use load_memory tool if you need to recall past conversations.",
  tools=[load_memory]
)

In [122]:
# updating the runner also
runner = Runner(
  agent=user_agent,
  app_name=APP_NAME,
  session_service=session_service,
  memory_service=memory_service,
)

App name mismatch detected. The runner is configured with app name "MemoryDemoApp", but the root agent was loaded from "/Users/liakooras/Desktop/UDEMIES/genAiKaggle/.conda/lib/python3.12/site-packages/google/adk/agents", which implies app name "agents".


In [123]:
await run_session(runner,"Whats my favourite football club?","club_conversation-3")


 ### Session:club_conversation-3

 User>Whats my favourite football club?


Model: > I see that your favorite team is PAOK Salonica.


In [124]:
await run_session(runner," When was the last time we won the league??","club_conversation-3")


 ### Session:club_conversation-3

 User> When was the last time we won the league??


Model: > I don't have information about when PAOK Salonica last won the league. My memory only contains our recent conversation about your favorite team and a motto.


In [125]:
memory_response = await memory_service.search_memory(user_id=USER_ID,app_name=APP_NAME,query="Whats my favourite football team?")

In [127]:
memory_response.memories

[MemoryEntry(content=Content(
   parts=[
     Part(
       text="My favourite moto is 'Nothing easy is worth doing', can you think of an alternative way of expressing it? "
     ),
   ],
   role='user'
 ), author='user', timestamp='2025-11-21T12:43:23.443404'),
 MemoryEntry(content=Content(
   parts=[
     Part(
       text='My Birdthay is on July 31.'
     ),
   ],
   role='user'
 ), author='user', timestamp='2025-11-21T12:43:26.143638'),
 MemoryEntry(content=Content(
   parts=[
     Part(
       text='My Birthday is on July 31.'
     ),
   ],
   role='user'
 ), author='user', timestamp='2025-11-21T12:43:55.395089'),
 MemoryEntry(content=Content(
   parts=[
     Part(
       text='My favourite team is PAOK Salonica.'
     ),
   ],
   role='user'
 ), author='user', timestamp='2025-11-21T13:06:31.397334')]

In [130]:
for memory in memory_response.memories:
  if memory.content and memory.content.parts:
    text = memory.content.parts[0].text
    print(f" [{memory.author}] : {text}")

 [user] : My favourite moto is 'Nothing easy is worth doing', can you think of an alternative way of expressing it? 
 [user] : My Birdthay is on July 31.
 [user] : My Birthday is on July 31.
 [user] : My favourite team is PAOK Salonica.


### Αutomating Memory Storage

##### Callbacks

ADK's callback system lets you hook into key execution moments. Callbacks are **Python functions** you define and attach to agents - ADK automatically calls them at specific stages, acting like checkpoints during the agent's execution flow.

**Think of callbacks as event listeners in your agent's lifecycle.**

**Available callback types:**

- `before_agent_callback` → Runs before agent starts processing a request
- `after_agent_callback` → Runs after agent completes its turn  
- `before_tool_callback` / `after_tool_callback` → Around tool invocations
- `before_model_callback` / `after_model_callback` → Around LLM calls
- `on_model_error_callback` → When errors occur

**Common use cases:**

- Logging and observability (track what the agent does)
- Automatic data persistence (like saving to memory)
- Custom validation or filtering
- Performance monitoring


In [137]:
#access the memory service and other runtime components

async def auto_save_to_memory(callback_context):
  """ Automatically save session to memory after each agent turn."""
  await callback_context._invocation_context.memory_service.add_session_to_memory(
    callback_context._invocation_context.session
  )

In [138]:
# agent with callback and preload memory tool

auto_memory_agent = LlmAgent(
  model=Gemini(model="gemini-2.5-flash-lite",retry_options=retry_config),
  name = "AutoMemoryAgent",
  instruction="Answer user questions.",
  tools=[preload_memory],
  after_agent_callback=auto_save_to_memory, #saves after each turn
)

In [139]:
auto_runner = Runner(
  agent = auto_memory_agent,
  app_name=APP_NAME,
  session_service=session_service,
  memory_service=memory_service
)

App name mismatch detected. The runner is configured with app name "MemoryDemoApp", but the root agent was loaded from "/Users/liakooras/Desktop/UDEMIES/genAiKaggle/.conda/lib/python3.12/site-packages/google/adk/agents", which implies app name "agents".


In [140]:
# Test 1: Tell the agent about a gift (first conversation)
# The callback will automatically save this to memory when the turn completes
await run_session(
    auto_runner,
    "I gifted a new toy to my nephew on his 1st birthday!",
    "auto-save-test",)


 ### Session:auto-save-test

 User>I gifted a new toy to my nephew on his 1st birthday!
Model: > That's wonderful! A 1st birthday is such a special milestone. I hope your nephew enjoys his new toy!


In [141]:
# Test 2: Ask about the gift in a NEW session (second conversation)
# The agent should retrieve the memory using preload_memory and answer correctly
await run_session(
    auto_runner,
    "What did I gift my nephew?",
    "auto-save-test-2",  # Different session ID - proves memory works across sessions!
)


 ### Session:auto-save-test-2

 User>What did I gift my nephew?
Model: > You gifted your nephew a new toy.
